Name:

Section:

Date:

<em><font size=4>**Instructions**</font></em>: Follow the tasks or answer the questions. Show the complete code how you arrived at the answers to the questions.

1. Import the Necessary Packages

In [1]:
# ---- Excel CSV ---- #
import pandas as pd

# ---- SQL ----- #
import sqlite3
from sqlalchemy import create_engine

# ---- JSON ----- #
import json

# ---- PDF ----- #
import pdfplumber

# ---- HTML ----- #
import requests
from bs4 import BeautifulSoup

# ---- Misc. ---- #
import os
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

2. Extract the necessary files


<font size="1px"><b>&nbsp;&nbsp;&nbsp;&nbsp;Optional: use OS to check</b></font>

&nbsp;&nbsp;2.1. Extract your data from the CSV and Excel

In [ ]:
csv_files = [x for x in os.listdir('./Datasets/') if x.endswith('.csv')]
excel_files = [x for x in os.listdir('./Datasets/') if x.endswith('.xlsx')]

display(csv_files, excel_files)

For CSV:

----

In [ ]:
csv = pd.read_csv('./Datasets/CleanedKaggleDataSets.csv')

In [ ]:
csv

----
<b><i> CSV File </b></i>

----

In [ ]:
csv

**Use the .info() method to find out how many entries there are.**

In [ ]:
csv.info()

**How many datasets are there that are related to the 'Heart'?**

In [ ]:
csv[csv['title'].str.contains('Heart', case=True)]['title'].nunique()

**Are there Null Values in the column Usability_index?**

In [ ]:
csv[csv['Usability_index'].isna()]

**Show all rows with a Usability_index of 10.0**

In [ ]:
csv[csv['Usability_index'].str.contains('Usability 10.0', na=False)]

**Get all rows with highest upvotes**

<font size='1px'><i>Bonus: Sort by Usability_index</i>

In [ ]:
csv[csv['up_vote'] == csv['up_vote'].max()].sort_values('Usability_index')

----
<b><i> Excel Files </b></i>

----

In [ ]:
sheets = pd.ExcelFile('./Datasets/OfficeData.xlsx')
sheets.sheet_names

In [ ]:
xlsx = pd.read_excel('./Datasets/OfficeData.xlsx', sheet_name=2)

In [ ]:
xlsx

**How many Males and Females are there?**

In [ ]:
xlsx['Gender'].value_counts()

**List all the Reasons of why some employee's are terminated**

In [ ]:
xlsx['Termination Reason'].unique()

**Show all rows with Unfair Dismissal as the reason of their Termination**

In [ ]:
xlsx[xlsx['Termination Reason'].str.contains('Unfair Dismissal', na=False)]

**In regards to the previous task, How many Males and Females are there?**

In [ ]:
xlsx[xlsx['Termination Reason'].str.contains('Unfair Dismissal', na=False)]['Gender'].value_counts()

**Show only the Employee, Position, and Department Columns**

<font size='1px'><i>Bonus: Sort by Employee</i>

In [ ]:
xlsx[['Employee','Position', 'Department']].sort_values('Employee')

----
&nbsp;&nbsp;2.2. Extract your data from the Database

In [ ]:
sql_files = [x for x in os.listdir('./Datasets/') if x.endswith('.db') or x.endswith('.sqlite3') or x.endswith('.sql')]
sql_files

In [ ]:
# --- Connect to the database --- #
'''
    For this portion, use the try - except method to 
    verify if you are connected to the database
'''


cnx = create_engine('sqlite:///Datasets/world.sqlite3')


**Show all the tables within the database**

In [ ]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', cnx)

**Show the cities table**

In [ ]:
db = pd.read_sql('select * from cities', cnx)
db

**Show the countries table**

In [ ]:
db_ = pd.read_sql('select * from countries', cnx)
db_

**Show the subregion table**

In [ ]:
db_1 = pd.read_sql('select * from subregions', cnx)
db_1

**Bonus:** Select the data from the "countries" table, specifically for the Philippines, without creating a Pandas DataFrame.

In [ ]:
pd.read_sql('SELECT * FROM countries WHERE name = "Philippines"', cnx)

----
<b><i> JSON File </b></i>

----

3. Extract the necessary files

<font size="1.5px"><b>&nbsp;&nbsp;&nbsp;&nbsp;Use OS to check the movies folder</b></font>

In [ ]:
path = './Datasets/movies/'

In [ ]:
movie_year = os.listdir(path)
movie_year

**Go to the year 2014**

In [ ]:
year = os.listdir(f'{path}/2014/')
year

**Without using pandas, open `gone-girl.json`**

In [ ]:
path2 = './Datasets/movies/2014/'

In [ ]:
with open(os.path.join(path2, 'gone-girl.json')) as f:
    gone_girl = json.load(f)
gone_girl

**Who are the actors in the movie?**

In [ ]:
gone_girl['actors']

**How long is the runtime and Whats the storyline of `gone girl` ?**

In [ ]:
for x in ['runtime', 'storyline']:
    print(f'Movie {x.title()}: {gone_girl.get(x)}')

**Create a list of dictionaries representing all movies across different years.**

In [ ]:
movie_list = list()

for years in tqdm(movie_year):
    for movies in os.listdir(os.path.join(f'{path}', f'{years}')):
        with open(os.path.join(f'{path}', f'{years}', f'{movies}'), encoding="ISO-8859-1") as movie_jsn:
            movie_list.append(json.load(movie_jsn))
        

In [ ]:
movie_list

**How many movies are there?**

In [ ]:
len(movie_list)

**Display all movies**

In [ ]:
movie_title = list()

for title in movie_list:
    movie_title.append(title['name'])
movie_title

**---------------------------- or this method ----------------------------**

In [ ]:
for title in movie_list:
    if ('name' in title.keys()):
        print(title['name'])

**Show all categories per movie**

In [ ]:
movie_categories = {}

for cat in movie_list:
    if 'name' in cat and 'categories' in cat:
        if cat['name'] in movie_categories:
            movie_categories[cat['name']].extend(cat['categories'])
        else:
            movie_categories[cat['name']] = cat['categories']

In [ ]:
movie_categories

**Show the storyline of each movie**

In [ ]:
movie_storyline = {}

for cat in movie_list:
    if 'name' in cat and 'storyline' in cat:
        if cat['name'] in movie_storyline:
            movie_storyline[cat['name']] = (cat['storyline'])
        else:
            movie_storyline[cat['name']] = cat['storyline']

**Get the Storyline for {Fave Movie}**

In [ ]:
movie_storyline['The Avengers']

----
<b><i> PDF File </b></i>

----

**Extract your PDF**

In [ ]:
with pdfplumber.open("./Datasets/sampledata_PDF.pdf") as pdf:
    tb = []
    for i in tqdm(range(len(pdf.pages))):
        page = ' '.join(pdf.pages[i].extract_text().split('\n')[1:-1])
        tb.append(page)

In [ ]:
tb

**Filter out the empty values**

In [ ]:
tb_filtered = [text for text in tb if text]  # This removes any empty or blank strings
tb_filtered

**How many pages are there?**

In [ ]:
len(tb_filtered)

----
<b><i> HTML </b></i>

----

**Create a session object using the request library**

In [2]:
session = requests.session()

**Grab your HTML**

In [3]:
url = 'https://medium.com/@CloverDX/8-dimensions-of-data-quality-8cc31648c530'

In [4]:
try: 
    """
        Fetch the Webpage
    """

    response = session.get(url)
    print(response.status_code,response.reason)

except response.status_code != '202':
    """
        Checks if the webpage was successfully fetched by looking at the status code
    """
    print(response.reason)

200 OK


**Parse the HTML Page and place it in a list**

In [16]:
soup = BeautifulSoup(response.text)
text = soup.find_all('h1')
text_list = [sentence.get_text(strip=True) for sentence in text][1:-1]
text_list

['The 8 dimensions of data quality',
 '1. Accuracy',
 '2. Consistency',
 '3. Relevancy',
 '4. Auditability',
 '5. Completeness',
 '6. Timeliness',
 '7. Validity',
 '8. Uniqueness']

In [17]:
h1_p_dict = {}

h1_tags = soup.find_all('h1')

for h1 in h1_tags:
    h1_text = h1.get_text(strip=True)
    p_tag = h1.find_next('p')
    
    if p_tag:
        p_text = p_tag.get_text(strip=True)
        h1_p_dict[h1_text] = p_text

In [18]:
h1_p_dict

{'8 dimensions of data quality': 'CloverDX',
 'The 8 dimensions of data quality': 'Accurate data reflects the real world. It’s factual and up to date, and serves as a source of reliable information that you can trust. When data is inaccurate, there are real-world implications.',
 '1. Accuracy': 'Accurate data reflects the real world. It’s factual and up to date, and serves as a source of reliable information that you can trust. When data is inaccurate, there are real-world implications.',
 '2. Consistency': 'Consistent data emerges when all instances are the same across multiple data sets. It’s important because it improves your ability to link data from multiple sources and thus increases the usability of the data.',
 '3. Relevancy': 'Data relevancy means different things for different industries. The data that is relevant to a financial institute will have little or no value to a healthcare provider. Similarly, data that a retailer collects to fuel its marketing efforts will have lit